# Radial sum and allow interactive display

This shows how to process the sparse data to produce:
 - diffraction patterns
 - a STEM image
 - a radial sum with interactive display


In [ ]:
%matplotlib widget

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import h5py

import stempy.image as stim
import stempy.io as stio

import ipywidgets as widgets
from ipywidgets import interact, interactive

In [ ]:
# User input
# Scan number, threshold
#crop_dimensions 2-tuple
#manual center? True False
#if manual input() center
#bf outer angle

In [ ]:
# Load a sparse 4D camera data set

plt.close('all')

scanNum = 31
threshold = 4.0

dPath = Path('/mnt/hdd1/2021.03.03')
fPath = Path('data_scan{}_th{}_electrons.h5'.format(scanNum, threshold))
print(dPath / fPath)

electron_events = stio.load_electron_counts(str(dPath / fPath)) # Scan dimension in stempy are (col, row)

print('Initial scan dimensions (col, row) = {}'.format(electron_events.scan_dimensions))

In [ ]:
# Remove flyback, crop to a smaller area (if desired)
crop_dimensions = (128, 128) # (col, row)

data_crop = electron_events.data.reshape(electron_events.scan_dimensions[::-1]) # reverese dimensions
data_crop = data_crop[0:crop_dimensions[0], 0:crop_dimensions[1]].ravel() # crop and ravel
print('Cropped scan dimensions = {}'.format(crop_dimensions))

In [ ]:
# Calculate a summed diffraction pattern
dp = stim.calculate_sum_sparse(data_crop[::10], electron_events.frame_dimensions)

# Set the center of the pattern
#center = (272, 304) # manual
center = stim.com_dense(dp) # center of intensity
print('Center of pattern at {0[0]}, {0[1]}'.format(center))

fg,ax = plt.subplots(1,1)
ax.imshow(np.log(dp + 0.1), vmax=None)
_ = ax.scatter(center[0], center[1], c='r')
ax.set(xlabel='x',ylabel='y');

In [ ]:
# Radial sum the sparse data
bf_outer_angle = 35

radial_sum = stim.radial_sum_sparse(data_crop, crop_dimensions, electron_events.frame_dimensions, center[::-1]) # center is (col,row)
bf = radial_sum[:,:,0:bf_outer_angle].sum(axis=2)

fg,ax = plt.subplots(1, 2)
ax[0].imshow(bf)
ax[0].set(title='Bright field')
ax[1].plot(radial_sum.sum(axis=(0,1)))
ax[1].set(title='Radial sum of all positions',xlabel='scattering angle (pixel^-1)')
fg.tight_layout()

In [ ]:
# Create interactive plot for BF and ADF images
fg1, ax1 = plt.subplots(1,1,figsize=(4, 4))
imax1 = ax1.imshow(radial_sum[:,:,0:50].sum(axis=2), vmin=0, vmax=1000, interpolation='none') # Set the initial image and intenstiy scaling 

def axUpdate(i):
    '''updates the plot'''
    s = np.sum(radial_sum[:,:,i[0]:i[1]],axis=2)
    imax1.set_data(s)
    imax1.set_clim(s.min(),s.max())

w1 = widgets.IntRangeSlider(
    value=[0, 50],
    min=0,
    max=288,
    step=2,
    description='VDF:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

interactive(axUpdate, i=w1)